In [1]:
import pandas as pd
import re
import html
import numpy as np

### Load and clean original data

In [2]:
df_act = pd.read_excel('Allosteric_interactions_BRENDA.xlsx', sheet_name='Activators', names=['EC', 'Enz', 'Met', 'Org', 'Mode'], usecols='A,B,C,E,I')
df_inh = pd.read_excel('Allosteric_interactions_BRENDA.xlsx', sheet_name='Inhibitors', names=['EC', 'Enz', 'Met', 'Org', 'Mode'], usecols='A,B,C,E,I')

#Remove metabolites named "additional information" and data from viruses, and limit species name to two names
df_act = df_act[~df_act['Org'].str.contains('virus')]
df_act = df_act[~df_act['Met'].str.contains('additional information')]
df_act['Org'] = df_act['Org'].apply(lambda x: ' '.join(x.split()[:2]))

df_inh.Org = df_inh.Org.fillna('')
df_inh = df_inh[~df_inh['Org'].str.contains('virus')]
df_inh = df_inh[~df_inh['Met'].str.contains('additional information')]
df_inh['Org'] = df_inh['Org'].apply(lambda x: ' '.join(x.split()[:2]))

#Save the cleaned dataframes to csv
df_act.to_csv('activators_clean.csv')
df_inh.to_csv('inhibitors_clean.csv')